# Code generator

In [1]:
import sys,os
import numpy as np
import sympy as sp
import pylab as py
from qgraf.qgraf import QGRAF
from form.form import FORM
from tools.tools import load
from scipy.integrate import quad
from sympy.parsing.sympy_parser import parse_expr
%matplotlib inline

In [2]:
def to_sympy(exp):
    ATOMS=[]
    exp=exp.replace('^','**')
    atoms=exp.replace('*',' ').replace('+',' ').replace('-',' ')
    atoms=atoms.replace('/',' ').replace('(',' ').replace(')',' ').split()
    atoms=sorted(set(atoms))
    for a in atoms:
        try: 
            int(a)
        except:
            ATOMS.append(a)
    ATOMS=sorted(set(ATOMS))
    for a in ATOMS:
        exec '%s=sp.Symbol("%s")'%(a,a)
    return parse_expr(exp)

## Create sympy symbols to be use across the notebook

In [3]:
CF=sp.S('CF')
gs2=sp.S('gs2')
gu2=sp.S('gu2')
gd2=sp.S('gd2')
Q2=sp.S('Q2')
s=sp.S('s')
s12=sp.S('s12')
t1=sp.S('t1')
u1=sp.S('u1')
t2=sp.S('t2')
u2=sp.S('u2')
qT2=sp.S('qT2')
xh=sp.S('xh')
zh=sp.S('zh')
z=sp.S('z')

## generate amps and sqamps with qgraf

- A, B, C and D now correspond to the figures in the paper


In [4]:
process={}
process['bornA']={'in':'A[p1],u[p2]','out':'u[q1],G[q2]'}
process['bornB']={'in':'A[p1],u[p2]','out':'G[q1],u[q2]'}
process['bornC']={'in':'A[p1],G[p2]','out':'u[q1],ub[q2]'}
process['bornD']={'in':'A[p1],G[p2]','out':'ub[q1],u[q2]'}

In [5]:
for k in process:
    conf={}                          
    conf['path2qgraf']='./qgraf/'          
    conf['label']=k           
    conf['style']='array.sty'        
    conf['model']='SM'               
    conf['in']=process[k]['in']        
    conf['out']=process[k]['out']       
    conf['loops']=0;                 
    conf['loop_momentum']=''     
    conf['extra']=[]
    conf['options']='notadpole'      
    process[k]['qgraf']=QGRAF(conf).data

In [6]:
for k in process: 
    for sqamp in process[k]['qgraf']['SQAMPS']: 
        print k,sqamp,':'
        print '\t',process[k]['qgraf']['SQAMPS'][sqamp]

bornB [a0*ca0] :
	(+1)*eps(A,p1,s1,MA,o1)*eps(G,q1,r1,MG,o2,j2)*(+1)*epsB(A,p1,s1,MA,o1001)*epsB(G,q1,r1,MG,o1002,j1002)*U(sc1,u,q2,r2,Mu)*UB(sc1,u,q2,r2,Mu)*(-i_)*gs^1*gI_(sc1,o2)*i_*(g_(sc1,p1)+g_(sc1,+p2)+g_(sc1)*Mu)*den(p1+p2,Mu)*(-i_)*gu^1*gI_(sc1,o1)*U(sc1,u,p2,s2,Mu)*UB(sc1,u,p2,s2,Mu)*(i_)*gu^1*gI_(sc1,o1001)*(-i_)*(g_(sc1,p1)+g_(sc1,+p2)+g_(sc1)*Mu)*den(p1+p2,Mu)*(i_)*gs^1*gI_(sc1,o1002)*Tr(j2,j1002)
bornB [a0*ca1] :
	(+1)*eps(A,p1,s1,MA,o1)*eps(G,q1,r1,MG,o2,j2)*(+1)*epsB(A,p1,s1,MA,o1001)*epsB(G,q1,r1,MG,o1002,j1002)*U(sc1,u,q2,r2,Mu)*UB(sc1,u,q2,r2,Mu)*(-i_)*gs^1*gI_(sc1,o2)*i_*(g_(sc1,p1)+g_(sc1,+p2)+g_(sc1)*Mu)*den(p1+p2,Mu)*(-i_)*gu^1*gI_(sc1,o1)*U(sc1,u,p2,s2,Mu)*UB(sc1,u,p2,s2,Mu)*(i_)*gs^1*gI_(sc1,o1002)*(-i_)*(g_(sc1,-p1)+g_(sc1,+q2)+g_(sc1)*Mu)*den(-p1+q2,Mu)*(i_)*gu^1*gI_(sc1,o1001)*Tr(j2,j1002)
bornB [a1*ca1] :
	(+1)*eps(A,p1,s1,MA,o1)*eps(G,q1,r1,MG,o2,j2)*(+1)*epsB(A,p1,s1,MA,o1001)*epsB(G,q1,r1,MG,o1002,j1002)*U(sc1,u,q2,r2,Mu)*UB(sc1,u,q2,r2,Mu)*(-i_)*gu^1*gI_

## use form to compute traces and convert expression into sympy expression

In [7]:
for k in process:
    process[k]['Pg']=None
    process[k]['Ppp']=None
    for kk in ['Pg','Ppp']:
        conf={}
        conf['label']=k
        conf['path2form']='./form'
        conf['qgraf']=process[k]['qgraf']
        conf['projection']='%s.frm'%kk
        terms=FORM(conf).data['total']
        combine=0
        for term in terms:
            term=to_sympy(term)
            term=term.replace('gs**2',gs2)
            term=term.replace('gu**2',gu2)
            term=term.replace('gd**2',gd2)
            term=term.replace(s12,s)
            term=term.replace(u2,t1)
            term=term.replace(t2,u1)
            combine+=term
        process[k][kk]=sp.simplify(combine)

## Display squared amplituds

In [8]:
for k in ['bornA','bornB','bornC','bornD']:
    for kk in ['Pg','Ppp']:
        print k,kk,':'
        print '\t',process[k][kk]

bornA Pg :
	8*ave*gs2*gu2*(Q2*s**2*(D**2*Q2 + D**2*s + D**2*u1 - 4*D*Q2 - 4*D*s - 4*D*u1 + 4*Q2 + 4*s + 4*u1) + Q2*t1**2*(D**2*Q2 + D**2*t1 + D**2*u1 - 4*D*Q2 - 4*D*t1 - 4*D*u1 + 4*Q2 + 4*t1 + 4*u1) + 2*s**2*t1**2*(D**2 - 6*D + 8) + s**2*t1*(3*D**2*Q2 + D**2*s - 16*D*Q2 - 4*D*s + 4*D*u1 + 20*Q2 + 4*s - 8*u1) + s*t1**2*(3*D**2*Q2 + D**2*t1 - 16*D*Q2 - 4*D*t1 + 4*D*u1 + 20*Q2 + 4*t1 - 8*u1) + 2*s*t1*(D**2*Q2**2 + D**2*Q2*u1 - 6*D*Q2**2 - 6*D*Q2*u1 + 2*D*u1**2 + 8*Q2**2 + 8*Q2*u1 - 4*u1**2))/(s**2*t1**2)
bornA Ppp :
	8*ave*gs2*gu2*u1*(-D*Q2**2 - 2*D*Q2*s - 2*D*Q2*u1 - D*s**2 - 2*D*s*u1 - D*u1**2 + 2*Q2**2 + 4*Q2*s + 4*Q2*u1 + 2*s**2 + 4*s*u1 + 2*u1**2)/t1**2
bornB Pg :
	8*ave*gs2*gu2*(Q2*s**2*(D**2*Q2 + D**2*s + D**2*t1 - 4*D*Q2 - 4*D*s - 4*D*t1 + 4*Q2 + 4*s + 4*t1) + Q2*u1**2*(D**2*Q2 + D**2*t1 + D**2*u1 - 4*D*Q2 - 4*D*t1 - 4*D*u1 + 4*Q2 + 4*t1 + 4*u1) + 2*s**2*u1**2*(D**2 - 6*D + 8) + s**2*u1*(3*D**2*Q2 + D**2*s - 16*D*Q2 - 4*D*s + 4*D*t1 + 20*Q2 + 4*s - 8*t1) + s*u1**2*(3*D**2*Q2 + D**

## Special treatment at LO.  
- We convert 4/3 to CF, 
- set strong coupling gs2 =1
- set charge factor gu2=gd2=1 
- save the results at sqamps
- convert s,t,u -> xh,zh,Q2,qT2

In [16]:
sqamps={}
for k in ['bornA','bornB','bornC','bornD']:
    sqamps[k]={}
    for kk in ['Pg','Ppp']:
        sqamps[k][kk]=process[k][kk].subs("D",sp.S(4))
        #sqamps[k][kk]=sqamps[k][kk]*CF/(sp.S(4)/sp.S(3))
        if k=='bornA' or k=='bornB': sqamps[k][kk]=sqamps[k][kk].subs("ave",1/sp.S(2*3))        
        if k=='bornC' or k=='bornD': sqamps[k][kk]=sqamps[k][kk].subs("ave",1/sp.S(2*8))
        sqamps[k][kk]=sqamps[k][kk].subs(gs2,1).subs(gu2,1)
        sqamps[k][kk]=sqamps[k][kk].subs(u1,-Q2-t1-s).simplify()
        sqamps[k][kk]=sqamps[k][kk].subs(s,Q2*(1/xh-1))
        sqamps[k][kk]=sqamps[k][kk].subs(t1,-Q2*(1-zh)-zh*qT2).simplify()

## Example

In [17]:
sqamps['bornA']['Pg']#.expand()

16*(2*Q2**2*xh*(-xh + 1) + Q2**2*(xh - 1)**2 + xh**2*(2*Q2**2 - 2*Q2*(-Q2*(zh - 1) + qT2*zh) + (-Q2*(zh - 1) + qT2*zh)**2))/(3*Q2*xh*(xh - 1)*(-Q2*(zh - 1) + qT2*zh))

## Construct LO partonic structure functions

In [18]:
#partialps=1/sp.S(8)/sp.pi**3/Q2/zh
ampave = lambda channel,projection: sqamps[channel][projection]#/(2*sp.pi)**3

##  Example

In [19]:
def convert(exp):
    sexp=str(exp)#.replace('sqrt','np.sqrt')
    #sexp=sexp.replace('log','np.log')
    sexp=sexp.replace('pi','np.pi')
    #sexp=sexp.replace('Abs','np.abs')
    #sexp=sexp.replace('polylog','fp.polylog')    
    return sexp

code=[]
code.append(r'#!/usr/bin/env python')
code.append(r'import numpy as np')
code.append(r'from mpmath import fp')
code.append(r'import numpy as np')

for _ in ['A','B','C','D']:
    code.append(r'def Pg%s(xh,zh,qT2,Q2):'%_)
    code.append('    return %s'%convert(ampave('born%s'%_,'Pg')))
for _ in ['A','B','C','D']:
    code.append(r'def Ppp%s(xh,zh,qT2,Q2):'%_)
    code.append('    return %s'%convert(ampave('born%s'%_,'Ppp')))

code=[l+'\n' for l in code]
F=open('LO.py','w')
F.writelines(code)
F.close()